In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import glm_model as gm
from glm_utils import *
import tensorflow as tf

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
#finally get rid of the ugly ass allen code. We import the pickled design 
#matrix stimulus, which is PCA projection of the images, and the downsampled
#traces. 

import pickle

pkl_file = open('./boc/formatted/LMcelldata')
data = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/LMstimdata')
stim = pickle.load(pkl_file)
pkl_file.close()

pkl_file = open('./boc/formatted/runspeed')
run_speed = pickle.load(pkl_file)
pkl_file.close()

In [3]:
#get the number of features, and the number of training examples. 
l, n =  stim.shape

In [7]:
from statsmodels.tsa.tsatools import lagmat
from sklearn.cross_validation import train_test_split
from scipy.stats import linregress
from itertools import product
import pyprind

import sys

n_pcs = range(1, 110, 5)
non_lin = [2]

alpha = 0
nlag = 2

#empty dictionary of the best model we've fit for each cell
best_model = {k: (-np.inf, None) for k in data.keys()}

#make a dictionary of dictionaries to store the results
scores_dict = {k: {} for k in data.keys()}

bar = pyprind.ProgBar(len(n_pcs)*len(non_lin))

for (l, n) in product(non_lin, n_pcs):
    bar.update()
    
    #lagged_stim = lagmat(stim[:, 0:n], maxlag = nlag, trim = None)[1:-1]       
    stim_array = np.concatenate((stim[:, 0:n], run_speed[:, np.newaxis]), axis = 1)
    
    
    for c in data.keys():
        X_train, X_test, y_train, y_test = train_test_split(stim_array, np.array(data[c]))     
        weight_init = np.linalg.pinv(X_train).dot(y_train)[:, np.newaxis]
       
        if l == 0:
            nonlin = tf.exp
        if l == 1:
            nonlin = tf.nn.sigmoid
        if l == 2:
            nonlin = tf.nn.relu

        
        model = gm.exponential_GLM(weight_init, non_lin = nonlin, verbose = False, alpha = 0, reg = 'l2', 
                                lr = 1e-4, train_params = False, scale_init = 1, bias_init = 0)
        
        loss, cv = model.fit(X_train, y_train[:, np.newaxis], X_test, y_test[:, np.newaxis], 
                             max_iters = 150) 
        
        
        w, o, s  = model.get_params()

    
        _, _, r2_test,_, _ = linregress(np.squeeze(model.predict_trace(X_test)), np.squeeze(y_test))
        _, _, r2_train,_, _ = linregress(np.squeeze(model.predict_trace(X_train)), np.squeeze(y_train))

        scores_dict[c][(l, n)] = (r2_test, r2_train) 

        if r2_test > best_model[c][0]:
            best_model[c] = (r2_test, (l, n))
            print "new best model for cell ", c, ". r2 test: ", r2_test


0%                  100%
[#                     ] | ETA: 00:00:00

new best model for cell  517442753 . r2 test:  0.0600365581453
new best model for cell  517444738 . r2 test:  -0.0020595088903
new best model for cell  517443331 . r2 test:  0.03090937433
new best model for cell  517442436 . r2 test:  0.0161703344833
new best model for cell  517445575 . r2 test:  -0.0115422822683
new best model for cell  517444426 . r2 test:  0.0069529278718
new best model for cell  517442471 . r2 test:  -0.0195923404058
new best model for cell  517443181 . r2 test:  0.0371471401969
new best model for cell  517443087 . r2 test:  -0.0255448082076
new best model for cell  517446833 . r2 test:  0.011737571097
new best model for cell  517443827 . r2 test:  -0.0208940650961
new best model for cell  517443924 . r2 test:  0.0149030320811
new best model for cell  517443957 . r2 test:  0.0571364806961
new best model for cell  517442722 . r2 test:  -0.00460570580491
new best model for cell  517446488 . r2 test:  -0.00331378209689
new best model for cell  517442675 . r2 test:  -0

[###                   ] | ETA: 01:17:56

 517443571 . r2 test:  0.00225825064867
new best model for cell  517444738 . r2 test:  0.0503344660597
new best model for cell  517445575 . r2 test:  0.0327831664402
new best model for cell  517444426 . r2 test:  0.0144343187149
new best model for cell  517442471 . r2 test:  0.00299776923381
new best model for cell  517443087 . r2 test:  0.0182500573983
new best model for cell  517442722 . r2 test:  0.0102891856079
new best model for cell  517446488 . r2 test:  0.0111004464332
new best model for cell  517442675 . r2 test:  0.0326792858182
new best model for cell  517443484 . r2 test:  0.0155342775388
new best model for cell  517443571 . r2 test:  0.0307171863237
new best model for cell  517442471 . r2 test:  0.0219983693793
new best model for cell  517446833 . r2 test:  0.0399863471086
new best model for cell  517443827 . r2 test:  -0.0162891002622
new best model for cell  517442722 . r2 test:  0.0161645806657
new best model for cell  517446554 . r2 test:  0.0180365702332
new best mode

[#####                 ] | ETA: 01:23:38

...

[####################  ] | ETA: 00:31:52

 517447069 . r2 test:  0.0483189947475
new best model for cell 

[######################] | ETA: 00:00:00

 517443087 . r2 test:  0.0404659385082
new best model for cell  517445575 . r2 test:  0.0830418227465
new best model for cell  517444426 . r2 test:  0.0443834143219
new best model for cell  517442471 . r2 test:  0.0537332004328
new best model for cell  517446554 . r2 test:  0.0180594820742



Total time elapsed: 05:54:39


In [15]:
for c in data.keys():
    
    if best_model[c][1] != None:
        (v, (l, n)) = best_model[c]
    
        print 'explained var: ', round(v, 3), 'non_lin:', l, 'n_components', n
        
    print best_model[c]

explained var:  0.205 non_lin: 0 n_components 1
(0.20471333787080059, (0, 1))
explained var:  0.067 non_lin: 0 n_components 1
(0.067399138643032402, (0, 1))
explained var:  -0.012 non_lin: 0 n_components 61
(-0.01225000620757246, (0, 61))
explained var:  0.321 non_lin: 0 n_components 1
(0.32066256818647759, (0, 1))
explained var:  0.243 non_lin: 0 n_components 21
(0.24282624633409033, (0, 21))
explained var:  0.174 non_lin: 0 n_components 1
(0.17381542853821436, (0, 1))
explained var:  0.047 non_lin: 0 n_components 1
(0.046700509571936236, (0, 1))
explained var:  0.012 non_lin: 0 n_components 81
(0.011955503314439762, (0, 81))
explained var:  0.112 non_lin: 0 n_components 1
(0.11185500602632083, (0, 1))
explained var:  0.263 non_lin: 0 n_components 21
(0.26327412441769821, (0, 21))
explained var:  0.005 non_lin: 0 n_components 61
(0.004968124413065044, (0, 61))
explained var:  0.182 non_lin: 0 n_components 1
(0.18216877669256215, (0, 1))
explained var:  0.118 non_lin: 0 n_components 1


In [ ]:
pkl_file = open('./boc/formatted/LNE_rs_relu', 'wb')
pickle.dump((best_model, scores_dict), pkl_file)
pkl_file.close()